## Project setup, imports, etc

In [39]:
# approach 2 - PRAW
import requests
import praw
import pandas as pd
import hashlib
import json
from datetime import datetime
from dateutil import tz
from collections import defaultdict

# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
USERNAME = 'malconst'
PASSWORD = '7(>vAn!GqC1"'
USER_AGENT = 'Seeker/0.0.1'

ModuleNotFoundError: No module named 'dbdict'

In [2]:
# Establish authorized Reddit instance
reddit = praw.Reddit(
    client_id = CLIENT_ID,
    client_secret = SECRET_TOKEN,
    user_agent = USER_AGENT,
    username = USERNAME,
    password = PASSWORD
)

## Search Reddit
PRAW Search doucmentation https://praw.readthedocs.io/en/latest/code_overview/models/subreddit.html?highlight=search#praw.models.Subreddit.search

*Important:* the search function in this implementation is a method on the Subreddit object, not Subreddits

For now, implemented hashing as described here https://www.codegrepper.com/code-examples/python/python+convert+string+to+hash

https://pythonspot.com/save-a-dictionary-to-a-file/

https://www.geeksforgeeks.org/how-to-read-dictionary-from-file-in-python/

https://stackoverflow.com/questions/2132985/how-to-import-or-include-data-structures-e-g-a-dict-into-a-python-file-from-a

In [225]:
# Variable setup and stuff
subreddit = reddit.subreddit("mechmarket") #set the subreddit object

# Access the database (local json object)
with open('dbdict.json') as f: #load it to f
    t = f.read() #read it to t
stored_search_test = json.loads(t) #turn t into a dictionary and store it

# Variable setup
current_hash_list = [] #to store current db hashes
searches = [] #holder for final storage; kept searches only
all_searches = [] #holder for analysis; all searches
search_queries = ["olivia","9009","hammerhead"] #list of the queries to search for (move to config.json)
limit = 3 #limit for the queries (move to config.json)
i_len = len(stored_search_test['searches']) #length of the current db searches

#store db hashes
for i in range(i_len):
    j_len = len(stored_search_test['searches'][i]['search_results']) #length of the j loop for some reason this works lol
    for j in range(j_len):
        current_hash_list.append(stored_search_test['searches'][i]['search_results'][j]['body']['hash']) #store the hash to the current_hash_list


In [ ]:
current_hash_list

In [223]:
# Search
for query in search_queries: #work through the list of queries
    results = [] #holder for handling the sets of search results stemming from each query
    for result in subreddit.search(query,sort="new", limit=limit): #make a hash in the same way as 
        # Structure the data before storing - there ahs to be a better way than this... like convering to JSON upstream
        print('evaluating result: ', result.id)
        result_id = str(result)
        to_hash = str(result.author)+ str(result.title)+ str(result.url)
        hash_object = hashlib.sha256(to_hash.encode('utf-8')) #
        hex_dig = hash_object.hexdigest() #the hash for this search result
        all_searches.append({'result_id': result_id, 'body': body}) #could delete this, don't really need it
        print('result hex_dig is: ', hex_dig)

        #Evaluate the new hash against the hash in the database search results
        
        # Check to see if this search result's hash is in the list. If yes ignore it, if it's not, add it to the db
        if hex_dig in current_hash_list:
            print('hex_dig is in the current hash list, breaking')
            break
        else:
            print('adding ', hex_dig)
            body = {
                'author':result.author.name,
                'title':result.title,
                'created date': datetime.fromtimestamp(result.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                'url':result.url,
                'hash':hex_dig
            }
            results.append({'result_id': result_id, 'body': body}) #just bundle the result_id and body into its own result object and add it to the the results list
    searches.append({'query':query,'search_results':results})
stored_search_test['searches'].append(searches) # update the stored_dictionary

# # store dictionary somewhere
# # Save dictionary
# json = json.dumps(stored_search_test) #the final updated database
# f = open("dbdict.json","w")
# f.write(json)
# f.close()

#35ed9de81a735a84299d6d96fd63d35e6f469813e2feed38fa547e5377762aeb


evaluating result:  u1i26n
result hex_dig is:  35ed9de81a735a84299d6d96fd63d35e6f469813e2feed38fa547e5377762aeb
adding  35ed9de81a735a84299d6d96fd63d35e6f469813e2feed38fa547e5377762aeb
evaluating result:  u1grab
result hex_dig is:  6fbd4e5a8bbebdd72223f4b6a99816026aba4c1518acc73b97c1944b51d2cefd
adding  6fbd4e5a8bbebdd72223f4b6a99816026aba4c1518acc73b97c1944b51d2cefd
evaluating result:  u1enhk
result hex_dig is:  b49fcf2038aa3cf49f20af487c8e80dd4afb11d937bb936ead87aa08b0022fd8
adding  b49fcf2038aa3cf49f20af487c8e80dd4afb11d937bb936ead87aa08b0022fd8
evaluating result:  u1k7h0
result hex_dig is:  18cda665689431943346060209e8ec2760c168512db5e1bbc95f19e0ab492e9e
adding  18cda665689431943346060209e8ec2760c168512db5e1bbc95f19e0ab492e9e
evaluating result:  u1k6po
result hex_dig is:  ec83c1f9196474ac00d542c60b6712dda140eb6cbb9fdf7906c8a2d96540e456
adding  ec83c1f9196474ac00d542c60b6712dda140eb6cbb9fdf7906c8a2d96540e456
evaluating result:  u1gng2
result hex_dig is:  67a0a74630cc2e36890e430fd514

In [224]:
print(stored_search_test)

{'searches': [{'query': 'olivia', 'search_results': [{'result_id': 'u0wzyl', 'body': {'author': 'shirouu-kun', 'title': '[US-CA] [H] Built Think6.5 v2 Pink Assassin, Think6.5 v2 Solder PCB, 2U Badges, Alpaca and Silent Alpaca Switches, JTK Night Sakura, Latrialum [W] Paypal', 'created date': '2022-04-10 20:13:00', 'url': 'https://www.reddit.com/r/mechmarket/comments/u0wzyl/usca_h_built_think65_v2_pink_assassin_think65_v2/', 'hash': '9278f1a8736b8f34f1cd26a36994a22cd527809a1bdd367f34482f69f32e4143'}}]}, {'query': '9009', 'search_results': [{'result_id': 'u08jd9', 'body': {'author': 'TradersParad1se', 'title': '[US-TX] [H] Keycult 2/65 Ocean Grey/ Silver, Tofu65 Grey Aluminum, Bauer 2 PC/Cyan, GMK Modo Light, GMK, GMK Modo2 [W] Paypal, GMK Redacted, GMK Dracula + Nightmode, GMK 9009', 'created date': '2022-04-09 20:52:43', 'url': 'https://www.reddit.com/r/mechmarket/comments/u08jd9/ustx_h_keycult_265_ocean_grey_silver_tofu65_grey/', 'hash': 'f673a1d4ea5b049c407723788a3086235a7e5ebf3ca4d1

## Graveyard

In [55]:
# #Approach 1 - 
# import requests
# import praw

# # note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
# CLIENT_ID = 'Mx4J1DKZ856aT1ug9glOdA'
# SECRET_TOKEN = 'j0a57wDUiC9TSDQhx1CYNz4jl65Aiw'
# USERNAME = 'malconst'
# PASSWORD = '7(>vAn!GqC1"'
# USER_AGENT = 'Seeker/0.0.1'

# auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_TOKEN)

# # here we pass our login method (password), username, and password
# data = {'grant_type': 'password',
#         'username': USERNAME,
#         'password': PASSWORD}

# # setup our header info, which gives reddit a brief description of our app
# headers = {'User-Agent': 'Seeker/0.0.1'}

# # send our request for an OAuth token
# res = requests.post('https://www.reddit.com/api/v1/access_token',
#                     auth=auth, data=data, headers=headers)

# # convert response to JSON and pull access_token value
# TOKEN = res.json()

# # add authorization to our headers dictionary
# headers = {**headers, **{'Authorization': f"bearer {TOKEN}"}}

# res = requests.get("https://oauth.reddit.com/r/python/hot",
#                    headers=headers).json()

# print(res.url)  # let's see what we get

